In [ ]:
import numpy as np # Import the numpy library and assign it the alias 'np'


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# You should have the Emotion detection project folder in your drive before running this code

In [ ]:
!cp '/content/drive/MyDrive/Emotion detection project/Imocap/IMOCAP/Copy of IEMOCAP_full_release_withoutVideos.tar.gz' /content/


In [ ]:
!tar -xzvf '/content/Copy of IEMOCAP_full_release_withoutVideos.tar.gz' -C /content/


Streaming output truncated to the last 5000 lines.
IEMOCAP_full_release/Session5/sentences/MOCAP_rotated/Ses05F_script03_1/Ses05F_script03_1_M030.txt
IEMOCAP_full_release/Session5/sentences/MOCAP_rotated/Ses05F_script03_1/Ses05F_script03_1_M021.txt
IEMOCAP_full_release/Session5/sentences/MOCAP_rotated/Ses05F_script03_1/Ses05F_script03_1_F007.txt
IEMOCAP_full_release/Session5/sentences/MOCAP_rotated/Ses05F_script03_1/Ses05F_script03_1_F003.txt
IEMOCAP_full_release/Session5/sentences/MOCAP_rotated/Ses05F_script03_1/Ses05F_script03_1_M033.txt
IEMOCAP_full_release/Session5/sentences/MOCAP_rotated/Ses05F_script03_1/Ses05F_script03_1_F013.txt
IEMOCAP_full_release/Session5/sentences/MOCAP_rotated/Ses05F_script03_1/Ses05F_script03_1_F022.txt
IEMOCAP_full_release/Session5/sentences/MOCAP_rotated/Ses05F_script03_1/Ses05F_script03_1_M012.txt
IEMOCAP_full_release/Session5/sentences/MOCAP_rotated/Ses05F_script03_1/Ses05F_script03_1_M016.txt
IEMOCAP_full_release/Session5/sentences/MOCAP_rotated/Ses0

In [ ]:
!pip install pydub pandas


# **Generating the continuous dataset**

##**this code generat two things:**
### 1.   folder contain all continuous speechs

###2.   Dataset contains:


*   the path to the speech
*   the equivalent emotions in (array)

*   the seconds which in the transition happen in (array)











In [ ]:
import os
import pandas as pd
from pydub import AudioSegment # Import AudioSegment from pydub
import contextlib # Import the contextlib module
import wave # Import the wave module


# Create an empty DataFrame with the required columns
df_results = pd.DataFrame(columns=['conti_path', 'emotions', 'time'])

sessions=['Session1','Session2','Session3','Session4','Session5']
csv_file = "/content/drive/MyDrive/Emotion detection project/Imocap/filtered_data.csv"
cv_ini="/content/drive/MyDrive/Emotion detection project/Imocap/iemocap_full_dataset.csv"

df_ini=pd.read_csv(cv_ini)
df = pd.read_csv(csv_file)
output_folder = "/content/continuous_speeches"  # Absolute path


os.makedirs(output_folder, exist_ok=True)
for filename in os.listdir(output_folder):
  file_path = os.path.join(output_folder, filename)
  try:
      if os.path.isfile(file_path) or os.path.islink(file_path):
          os.unlink(file_path)  # Remove files or symbolic links
      elif os.path.isdir(file_path):
          shutil.rmtree(file_path)  # Remove subdirectories
  except Exception as e:
      print(f"Failed to delete {file_path}. Reason: {e}")
# Example wav file path to
for session in sessions:

# Define the base path
  base_path = f"/content/IEMOCAP_full_release/{session}/sentences/wav/"

# Loop over all subfolders in the base path
  c=0
  for folder_name in os.listdir(base_path):
      folder_path = os.path.join(base_path, folder_name)


      if os.path.isdir(folder_path):  # Check if it's a directory

          combined = AudioSegment.empty()
          emotions=[]
          timestamp=[]
          duration=0
          for wav_file in os.listdir(folder_path):
            if wav_file.endswith(".wav"):
              wav_file_path = os.path.join(folder_path, wav_file)

              emotion_ini = df_ini.loc["/content/IEMOCAP_full_release/" + df_ini['path']  == wav_file_path, 'emotion']

              emotion = df.loc[df['path'] == wav_file_path, 'emotion']
              if not emotion.empty:

                emotions.append(emotion.values[0])
                audio = AudioSegment.from_wav(wav_file_path)
                combined += audio
                with contextlib.closing(wave.open(wav_file_path, 'r')) as wf:
                    frames = wf.getnframes()
                    rate = wf.getframerate()
                    duration += frames / float(rate)


                    timestamp.append(duration)


          output_file = os.path.join(output_folder, f"combined_speech_{folder_name}.wav")
          combined.export(output_file, format="wav")


          new_entry = {'conti_path': output_file, 'emotions': emotions, 'time': timestamp}
          df_results = pd.concat([df_results, pd.DataFrame([new_entry])], ignore_index=True)



df_results.head()
df_results.to_csv('/content/merged_emotions.csv', index=False)



# **Download the the folder that contain the continuous IMOCAP speechs in comprised zib file**

In [ ]:
import shutil

# Replace 'folder_name' with the name of your folder
folder_name = "continuous_speeches"
shutil.make_archive(folder_name, 'zip', folder_name)
from google.colab import files

# Replace 'your_folder_name.zip' with the name of your zipped file
files.download(f"{folder_name}.zip")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>